In [1]:
import re
import logging
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import math
import time
import pickle
from collections import defaultdict
from data.configs.demo import config
from collections import defaultdict

from transformers import AutoTokenizer, AutoModel, BertModel, BertConfig, AutoModelForSequenceClassification
from utils.Manager import Manager

from models.Embeddings.BERT import BERT_Embedding
from models.Encoders.CNN import CNN_Encoder,CNN_User_Encoder
from models.Encoders.RNN import RNN_Encoder,RNN_User_Encoder
from models.Encoders.MHA import MHA_Encoder, MHA_User_Encoder
from models.Modules.DRM import Matching_Reducer, Identical_Reducer
from models.Rankers.BERT import BERT_Onepass_Ranker, BERT_Original_Ranker
from models.Rankers.CNN import CNN_Ranker
from models.Encoders.Pooling import Attention_Pooling, Average_Pooling
from models.UniLM.modeling import TuringNLRv3Model, TuringNLRv3ForSequenceClassification, relative_position_bucket
from models.UniLM.configuration_tnlrv3 import TuringNLRv3Config

from models.BaseModel import BaseModel

from models.Encoders.BERT import BERT_Encoder
from models.Encoders.Pooling import *

from models.ESM import ESM
from models.TESRec import TESRec
 
from models.Modules.Attention import MultiheadAttention, get_attn_mask, XSoftmax
torch.set_printoptions(threshold=100000)

In [2]:
# m = AutoModel.from_pretrained('bert-base-uncased',cache_dir=config.path + 'bert_cache/')
# m2 = AutoModel.from_pretrained('microsoft/deberta-base',cache_dir=config.path + 'bert_cache/')
# m3 = TuringNLRv3ForSequenceClassification.from_pretrained(config.unilm_path, config=TuringNLRv3Config.from_pretrained(config.unilm_config_path))

# t = AutoTokenizer.from_pretrained('bert-base-uncased', cache_dir=config.path + "bert_cache/")
# t2 = AutoTokenizer.from_pretrained('microsoft/deberta-base', cache_dir=config.path + "bert_cache/")

In [3]:
# config.reducer = 'entity'
# config.embedding = 'deberta'
# config.bert = 'microsoft/deberta-base'
# config.device = 0
config.seed = None
manager = Manager(config)

manager.hidden_dim = 768
manager.bert = 'deberta'
manager.granularity = 'token'

loaders = manager.prepare()
X1 = list(loaders[0])
X2 = list(loaders[1])
x1 = X1[0]
x2 = X2[0]

[2021-10-23 10:13:04,437] INFO (utils.Manager) Hyper Parameters are 
{
    "scale": "demo",
    "mode": "train",
    "batch_size": 5,
    "batch_size_news": 100,
    "batch_size_history": 100,
    "k": 5,
    "threshold": -Infinity,
    "abs_length": 40,
    "signal_length": 100,
    "his_size": 50,
    "cdd_size": 5,
    "impr_size": 10,
    "dropout_p": 0.2,
    "lr": 0.0001,
    "bert_lr": 3e-05,
    "embedding": "bert",
    "encoderN": "cnn",
    "encoderU": "rnn",
    "selector": "sfi",
    "reducer": "matching",
    "ranker": "onepass",
    "pooler": "attn",
    "bert_dim": 768,
    "hidden_dim": 768,
    "base_rank": 0,
    "world_size": 0,
    "seed": null,
    "granularity": "token",
    "debias": true,
    "full_attn": true,
    "descend_history": false,
    "shuffle_pos": false,
    "save_pos": false,
    "sep_his": false,
    "diversify": false,
    "no_dedup": false,
    "no_order_embed": false,
    "no_rm_punc": false,
    "fast": false,
    "scheduler": "linear",
    "wa

In [ ]:
embedding = BERT_Embedding(manager)

# encoderN = CNN_Encoder(manager)
# encoderN = RNN_Encoder(manager)
# encoderN = MHA_Encoder(manager)

# encoderU = CNN_User_Encoder(manager)
encoderU = RNN_User_Encoder(manager)
# encoderU = MHA_User_Encoder(manager)
# encoderU = Attention_Pooling(manager)
# encoderU = Average_Pooling(manager)

# reducer = Matching_Reducer(manager)
# reducer = Identical_Reducer(manager)

# ranker = CNN_Ranker(manager)
# ranker = BERT_Onepass_Ranker(manager)
# ranker = BERT_Original_Ranker(manager)

# model = TESRec(manager, embedding, encoderN, encoderU, reducer).to(manager.device)
# model = ESM(manager, embedding, encoderN, encoderU, reducer, ranker).to(manager.device)

# manager.load(model, 589, strict=False)

In [4]:
from transformers.modeling_utils import apply_chunking_to_forward
from transformers.activations import ACT2FN

class BertSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query = nn.Linear(config.hidden_size, self.all_head_size)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)

        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)

    def transpose_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward(
        self,
        hidden_states,
        attention_mask=None
    ):
        mixed_query_layer = self.query(hidden_states)

        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(self.key(hidden_states))
        value_layer = self.transpose_for_scores(self.value(hidden_states))

        # Take the dot product between "query" and "key" to get the raw attention scores.
        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        if attention_mask is not None:
            attention_probs = XSoftmax.apply(attention_scores, attention_mask, -1)

        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        attention_probs = self.dropout(attention_probs)

        context_layer = torch.matmul(attention_probs, value_layer)

        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)

        outputs = (context_layer,)

        return outputs


class BertSelfOutput(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, hidden_states, input_tensor):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states


class BertAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.self = BertSelfAttention(config)
        self.output = BertSelfOutput(config)

    def forward(
        self,
        hidden_states,
        attention_mask=None,
    ):
        self_outputs = self.self(
            hidden_states,
            attention_mask,
        )
        attention_output = self.output(self_outputs[0], hidden_states)
        outputs = (attention_output,)
        return outputs


class BertIntermediate(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.intermediate_size)
        if isinstance(config.hidden_act, str):
            self.intermediate_act_fn = ACT2FN[config.hidden_act]
        else:
            self.intermediate_act_fn = config.hidden_act

    def forward(self, hidden_states):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.intermediate_act_fn(hidden_states)
        return hidden_states


class BertOutput(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.intermediate_size, config.hidden_size)
        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, hidden_states, input_tensor):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states


class BertLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.chunk_size_feed_forward = config.chunk_size_feed_forward
        self.seq_len_dim = 1
        self.attention = BertAttention(config)
        self.intermediate = BertIntermediate(config)
        self.output = BertOutput(config)

    def forward(
        self,
        hidden_states,
        attention_mask=None,
    ):
        self_attention_outputs = self.attention(
            hidden_states,
            attention_mask
        )
        attention_output = self_attention_outputs[0]

        outputs = self_attention_outputs[1:]

        layer_output = apply_chunking_to_forward(
            self.feed_forward_chunk, self.chunk_size_feed_forward, self.seq_len_dim, attention_output
        )
        outputs = (layer_output,) + outputs

        return outputs

    def feed_forward_chunk(self, attention_output):
        intermediate_output = self.intermediate(attention_output)
        layer_output = self.output(intermediate_output, attention_output)
        return layer_output


In [5]:
from transformers import BertConfig
config = BertConfig()
a = BertLayer(config)

In [9]:
input = torch.rand((2,10,768))
attn_mask = torch.ones((2,1,1,10))
res = a(input, attn_mask)

In [12]:
res[0].shape

torch.Size([2, 10, 768])